# PREDICTION OF AMAZON STOCKS

## Uploading data

In [1]:
import pandas as pd
import numpy as np

In [3]:
data_raw = pd.read_csv("/Users/phuonglucydoan/Desktop/STUDY/Spring 2020/Courses/IMSE 8370 _ Supply Chain Modeling and Analysis/Individual Project/Stock Price Prediction/AMZN.csv"
                  , index_col="Date")

In [4]:
data_raw.describe()

,Open,High,Low,Close,Adj Close,Volume
count,5120.000000,5120.000000,5120.000000,5120.000000,5120.000000,5.120000e+03
mean,390.051718,394.238784,385.457587,390.075649,390.075649,6.532767e+06
std,558.539251,563.943783,552.390044,558.477764,558.477764,5.174871e+06
min,5.910000,6.100000,5.510000,5.970000,5.970000,8.813000e+05
25%,40.939374,41.740002,40.147500,40.995000,40.995000,3.637375e+06
50%,126.005001,127.594997,123.815003,125.959999,125.959999,5.410350e+06
75%,399.102501,403.730003,395.845002,399.302505,399.302505,7.796250e+06
max,2443.199951,2475.000000,2396.010010,2474.000000,2474.000000,1.043292e+08


In [5]:
data_DJIA = pd.read_csv("/Users/phuonglucydoan/Desktop/STUDY/Spring 2020/Courses/IMSE 8370 _ Supply Chain Modeling and Analysis/Individual Project/Stock Price Prediction/DJI.csv"
                  , index_col="Date")

In [6]:
data_raw['DJIA_Open']=data_DJIA['Open']

In [7]:
data_raw.head(5)

,Open,High,Low,Close,Adj Close,Volume,DJIA_Open
Date,,,,,,,
2000-01-03,81.5000,89.5625,79.046875,89.3750,89.3750,16117600,11501.849609
2000-01-04,85.3750,91.5000,81.750000,81.9375,81.9375,17487400,11349.750000
2000-01-05,70.5000,75.1250,68.000000,69.7500,69.7500,38457400,10989.370117
2000-01-06,71.3125,72.6875,64.000000,65.5625,65.5625,18752000,11113.370117
2000-01-07,67.0000,70.5000,66.187500,69.5625,69.5625,10505400,11247.059570


## Generating features

In [8]:
#Create sub-function that generates 6 original features

def add_original_feature (df, df_new):
    df_new["open"] = df["Open"]
    df_new["open_1"] = df["Open"].shift(1)
    df_new["close_1"] = df["Close"].shift(1)
    df_new["high_1"] = df["High"].shift(1)
    df_new["low_1"] = df["Low"].shift(1)
    df_new["volume_1"] = df["Volume"].shift(1)

In [9]:
#Create sub-function that generates features related to average close prices

def add_avg_price (df, df_new):
    df_new["avg_price_5"] = df["Close"].rolling(5).mean().shift(1)
    df_new["avg_price_30"] = df["Close"].rolling(21).mean().shift(1)
    df_new["avg_price_365"] = df["Close"].rolling(252).mean().shift(1)
    df_new["ratio_avg_price_5_30"] = df_new["avg_price_5"] / df_new["avg_price_30"]
    df_new["ratio_avg_price_5_365"] = df_new["avg_price_5"] / df_new["avg_price_365"]
    df_new["ratio_avg_price_30_365"] = df_new["avg_price_30"] / df_new["avg_price_365"]

In [10]:
#Create sub-function that generates features related to average volumes

def add_avg_volume (df, df_new):
    df_new["avg_volume_5"] = df["Volume"].rolling(5).mean().shift(1)
    df_new["avg_volume_30"] = df["Volume"].rolling(21).mean().shift(1)
    df_new["avg_volume_365"] = df["Volume"].rolling(252).mean().shift(1)
    df_new["ratio_avg_volume_5_30"] = df_new["avg_volume_5"] / df_new["avg_volume_30"]
    df_new["ratio_avg_volume_5_365"] = df_new["avg_volume_5"] / df_new["avg_volume_365"]
    df_new["ratio_avg_volume_30_365"] = df_new["avg_volume_30"] / df_new["avg_volume_365"]

In [11]:
#Create sub-function that generates features related to standard deviation of close prices

def add_std_price (df, df_new):
    df_new["std_price_5"] = df["Close"].rolling(5).std().shift(1)
    df_new["std_price_30"] = df["Close"].rolling(21).std().shift(1)
    df_new["std_price_365"] = df["Close"].rolling(252).std().shift(1)
    df_new["ratio_stdg_price_5_30"] = df_new["std_price_5"] / df_new["std_price_30"]
    df_new["ratio_std_price_5_365"] = df_new["std_price_5"] / df_new["std_price_365"]
    df_new["ratio_std_price_30_365"] = df_new["std_price_30"] / df_new["std_price_365"]

In [12]:
#Create sub-function that generates features related to standard deviation of volumes

def add_std_volume (df, df_new):
    df_new["std_volume_5"] = df["Volume"].rolling(5).std().shift(1)
    df_new["std_volume_30"] = df["Volume"].rolling(21).std().shift(1)
    df_new["std_volume_365"] = df["Volume"].rolling(252).std().shift(1)
    df_new["ratio_stdg_volume_5_30"] = df_new["std_volume_5"] / df_new["std_volume_30"]
    df_new["ratio_std_volume_5_365"] = df_new["std_volume_5"] / df_new["std_volume_365"]
    df_new["ratio_std_volume_30_365"] = df_new["std_volume_30"] / df_new["std_volume_365"]

In [13]:
weights = np.arange(1,6)
weights

array([1, 2, 3, 4, 5])

In [14]:
#Create sub-function that generates 9 return-based features

def add_return_feature (df, df_new):
    df_new["return_1"] = ((df["Close"] - df["Close"].shift(1))/df["Close"].shift(1)).shift(1)
    df_new["return_5"] = ((df["Close"] - df["Close"].shift(5))/df["Close"].shift(5)).shift(1)
    df_new["return_30"] = ((df["Close"] - df["Close"].shift(21))/df["Close"].shift(21)).shift(1)
    df_new["return_365"] = ((df["Close"] - df["Close"].shift(252))/df["Close"].shift(252)).shift(1)
    df_new["moving_avg_5"] = df_new["return_1"].rolling(5).mean().shift(1)
    df_new["moving_avg_30"] = df_new["return_1"].rolling(21).mean().shift(1)
    df_new["moving_avg_365"] = df_new["return_1"].rolling(255).mean().shift(1)
    df_new["weighted_moving_avg_5"] = df_new["return_1"].rolling(5).apply(lambda wma_return: np.dot(wma_return, weights)/weights.sum(), raw=True)
    df_new["exp_smoothing_5"] = df_new["moving_avg_5"].ewm(span=5).mean().shift(1)

In [15]:
#The main feature generating function that combines all sub-functions

def generate_feature(df):
    df_new=pd.DataFrame()
    add_original_feature(df, df_new)
    add_avg_price(df, df_new)
    add_avg_volume(df, df_new)
    add_std_price(df, df_new)
    add_std_volume(df, df_new)
    add_return_feature(df, df_new)
    
    df_new = df_new.dropna(axis=0)

    df_new["close"] = df["Close"]
    df_new["DJIA_open"] = df["DJIA_Open"]
    return df_new

In [16]:
data = generate_feature(data_raw)

In [17]:
data.describe()

,open,open_1,close_1,high_1,low_1,volume_1,avg_price_5,avg_price_30,avg_price_365,ratio_avg_price_5_30,...,return_5,return_30,return_365,moving_avg_5,moving_avg_30,moving_avg_365,weighted_moving_avg_5,exp_smoothing_5,close,DJIA_open
count,4863.000000,4863.000000,4863.000000,4863.000000,4863.000000,4.863000e+03,4863.000000,4863.000000,4863.000000,4863.000000,...,4863.000000,4863.000000,4863.000000,4863.000000,4863.000000,4863.000000,4863.000000,4863.000000,4863.000000,4863.000000
mean,408.234555,407.749730,407.777036,412.054179,403.019954,6.414610e+06,406.817873,402.969909,358.683942,1.008573,...,0.007330,0.029061,0.347583,0.001476,0.001438,0.001278,0.001480,0.001474,408.263294,14424.742829
std,567.320847,566.649208,566.576964,572.173222,560.363765,5.101930e+06,565.137865,559.357005,505.230039,0.056227,...,0.066856,0.130211,0.559432,0.013219,0.006039,0.001723,0.014651,0.009905,567.254066,5463.893193
min,5.910000,5.910000,5.970000,6.100000,5.510000,8.813000e+05,6.122000,7.097619,11.652123,0.786490,...,-0.327749,-0.493789,-0.869728,-0.073367,-0.030202,-0.005809,-0.088693,-0.042938,5.970000,6547.009766
25%,41.955000,41.929998,42.105002,42.799999,41.285000,3.559800e+06,42.120999,42.404761,40.567579,0.978321,...,-0.024238,-0.044804,0.030688,-0.004657,-0.001893,0.000479,-0.005161,-0.003505,42.135000,10414.839844
50%,145.169998,144.880005,145.750000,147.169998,143.559998,5.320200e+06,144.440002,139.639047,123.203611,1.010410,...,0.005634,0.026961,0.269287,0.001310,0.001451,0.001258,0.001223,0.001256,145.820007,12417.959961
75%,446.630005,444.414993,445.544998,449.845002,439.615006,7.683050e+06,439.803006,434.776195,355.935021,1.036140,...,0.034921,0.091908,0.608031,0.007056,0.004455,0.002241,0.007794,0.005945,450.779999,17590.355469
max,2443.199951,2443.199951,2474.000000,2475.000000,2396.010010,1.043292e+08,2389.402002,2319.009545,1893.075991,1.329004,...,0.676225,1.090476,2.701399,0.115353,0.039045,0.005950,0.121268,0.084940,2474.000000,29440.470703


In [18]:
data.round(decimals=3).head(5)

,open,open_1,close_1,high_1,low_1,volume_1,avg_price_5,avg_price_30,avg_price_365,ratio_avg_price_5_30,...,return_5,return_30,return_365,moving_avg_5,moving_avg_30,moving_avg_365,weighted_moving_avg_5,exp_smoothing_5,close,DJIA_open
Date,,,,,,,,,,,,,,,,,,,,,
2001-01-09,14.562,14.438,14.938,15.562,13.750,10444700.0,15.288,18.765,45.344,0.815,...,-0.040,-0.368,-0.785,-0.025,-0.022,-0.005,-0.003,-0.005,16.375,10625.21
2001-01-10,15.844,14.562,16.375,16.625,14.562,23455200.0,15.788,18.527,45.135,0.852,...,0.180,-0.234,-0.763,0.001,-0.017,-0.005,0.029,-0.012,16.500,10568.48
2001-01-11,15.750,15.844,16.500,17.000,15.562,11002500.0,15.575,18.196,44.935,0.856,...,-0.060,-0.296,-0.753,0.042,-0.008,-0.004,0.017,-0.008,17.000,10600.20
2001-01-12,17.344,15.750,17.000,17.312,15.625,12756600.0,15.875,17.801,44.750,0.892,...,0.097,-0.328,-0.733,-0.010,-0.012,-0.003,0.031,0.009,17.688,10608.74
2001-01-16,18.312,17.344,17.688,18.500,17.125,8434200.0,16.500,17.411,44.559,0.948,...,0.215,-0.316,-0.732,0.020,-0.015,-0.003,0.038,0.003,18.062,10525.78


## Data Splitting and Pre-processing

In [19]:
#Splitting data and counting training samples

start_train = '2001-01-09'
end_train = '2018-12-31'
start_test = '2019-01-02'
end_test = '2020-05-08'
data_train = data.loc[start_train:end_train]
X_train = data_train.drop('close', axis=1).values
y_train = data_train['close'].values

print(X_train.shape, y_train.shape)

(4522, 40) (4522,)


In [20]:
#Counting testing samples

data_test = data.loc[start_test:end_test]
X_test = data_test.drop('close', axis=1).values
y_test = data_test['close'].values

print(X_test.shape)

(341, 40)


In [21]:
#Rescaling both training and testing datasets

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_scaled_train = scaler.fit_transform(X_train)
X_scaled_test = scaler.transform(X_test)

## Learning from Datasets and Making Predictions

In [22]:
#Install predicting algorithms and performance-evaluating metrics

from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

### SGD-based Linear Regression

In [23]:
#Find out optimal set of parameters for SGD-based Linear Regression model

param_grid = {
    "alpha": [1e-5, 3e-5, 1e-4],
    "eta0": [0.01, 0.03, 0.1],
}
lr = SGDRegressor(loss='squared_loss', learning_rate='constant', penalty='l2', max_iter=1000)
grid_search = GridSearchCV(lr, param_grid, cv=5, scoring='r2')
grid_search.fit(X_scaled_train, y_train)

print(grid_search.best_params_)

{'alpha': 1e-05, 'eta0': 0.01}


In [24]:
#Use the optimal model to make predictions of the testing samples

lr_best = grid_search.best_estimator_
predictions_lr = lr_best.predict(X_scaled_test)

In [25]:
#Measure predicting performance of Linear Regression model

MSE_lr = mean_squared_error(y_test, predictions_lr).round(decimals=3)
MAE_lr = mean_absolute_error(y_test, predictions_lr).round(decimals=3)
Rsquared_lr = r2_score(y_test, predictions_lr).round(decimals=3)
print("For Linear Regression model,",
      "Mean squared error:", MSE_lr, ", Mean absolute error:", MAE_lr, ", R_squared:", Rsquared_lr)

For Linear Regression model, Mean squared error: 1272.451 , Mean absolute error: 24.983 , R_squared: 0.956


### Regression Forest

In [26]:
#Find out optimal set of parameters for Regression Forest model

param_grid = {
    'max_depth': [50,70,80],
    'min_samples_split': [5,10],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [3,5]
}
rf = RandomForestRegressor(n_estimators=500, n_jobs=-1)
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

{'max_depth': 50, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 10}


In [81]:
#Use the optimal model to make predictions of the testing samples

rf_best = grid_search.best_estimator_
predictions_rf = rf_best.predict(X_test)

In [82]:
#Measure predicting performance of Regression Forest model

MSE_rf = mean_squared_error(y_test, predictions_rf).round(decimals=3)
MAE_rf = mean_absolute_error(y_test, predictions_rf).round(decimals=3)
Rsquared_rf = r2_score(y_test, predictions_rf).round(decimals=3)
print("For Regression Forest model,",
      "Mean squared error:", MSE_rf, ", Mean absolute error:", MAE_rf, ", R_squared:", Rsquared_rf)

For Regression Forest model, Mean squared error: 41862.093 , Mean absolute error: 149.186 , R_squared: -0.434
